<a href="https://colab.research.google.com/github/NiyaziOnurYantira/FineTuning/blob/main/mistral-instruct/mistral_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip installs

!pip install -q datasets peft requests torch bitsandbytes transformers trl accelerate sentencepiece matplotlib

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import torch.nn.functional as F
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
from datasets import load_dataset, Dataset, DatasetDict
from datetime import datetime
from peft import PeftModel
import matplotlib.pyplot as plt

In [ ]:
# Constants

BASE_MODEL =  "mistralai/Mistral-7B-Instruct-v0.3"
PROJECT_NAME = "medical-bot-Mistral-7B-Instruct-v0.3"
HF_USER = "OnurYantira" # your HF name here! Or use mine if you just want to reproduce my results.

# The run itself

RUN_NAME = "2025-04-07_08.44.40"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = None # or REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"

# Uncomment this line if you wish to use my model
# FINETUNED_MODEL = f"ed-donner/{PROJECT_RUN_NAME}"

# Data

DATASET_NAME = f"ruslanmv/ai-medical-chatbot"
# Or just use the one I've uploaded
# DATASET_NAME = "ed-donner/pricer-data"

# Hyperparameters for QLoRA

QUANT_4_BIT = True

%matplotlib inline

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}

In [ ]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']

In [ ]:
train = train.shuffle(seed=42).select(range(5000))
test_indices = train[4000:5000]  # İlk 1000 satır test olacak
train_indices = train[:4000]

# Test ve eğitim veri kümesini belirleyecek indeksleri oluştur
test_indices = list(range(4000, 5000))
train_indices = list(range(4000))  # 0'dan 3999'a kadar olan indeksler

# Yeni veri kümelerini oluştur
test = train.select(test_indices)
train = train.select(train_indices)

# Sonuçları kontrol et
print(f"Yeni Train Dataset Boyutu: {len(train)}")
print(f"Yeni Test Dataset Boyutu: {len(test)}")

In [ ]:
# pick the right quantization (thank you Robert M. for spotting the bug with the 8 bit version!)

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
QUANT_4_BIT = True

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

# Load the fine-tuned model with PEFT
if REVISION:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL, revision=REVISION)
else:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)


print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

In [ ]:
from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

prompt = "My stomach has been hurting after eating dairy. What should I do?"

chat = [
    {"role": "user", "content": prompt}
]

# ChatML formatına çevir
formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

# Modelden çıktı al
output = fine_tuned_model.generate(**inputs, max_new_tokens=200, streamer=streamer)


In [ ]:


from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

prompt = "After taking antibiotics for a week, I started having diarrhea. Is this normal?"

chat = [
    {"role": "user", "content": prompt}
]

# ChatML formatına çevir
formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

# Modelden çıktı al
output = fine_tuned_model.generate(**inputs, max_new_tokens=200, streamer=streamer)


In [ ]:


from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

prompt = "Interpret this lab result: Hemoglobin = 10.2 g/dL, WBC = 14.5 x10^9/L"

chat = [
    {"role": "user", "content": prompt}
]

# ChatML formatına çevir
formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

# Modelden çıktı al
output = fine_tuned_model.generate(**inputs, max_new_tokens=200, streamer=streamer)


In [ ]:


from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

prompt = "Can I take ibuprofen and paracetamol together?"

chat = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."},
    {"role": "user", "content": prompt}
]

# ChatML formatına çevir
formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

# Modelden çıktı al
output = fine_tuned_model.generate(**inputs, max_new_tokens=200, streamer=streamer)


In [ ]:
# Hasta-Doktor Diyaloğu örneği:
chat_dialogue = [
    {"role": "user", "content": "I've been experiencing severe headaches and blurred vision. What should I do?"},
    {"role": "assistant", "content": "Based on your symptoms, I recommend you get an eye exam immediately and consult a neurologist."}
]

# Chat template uyguluyoruz. Eğitim sırasında generation prompt eklemiyoruz, çünkü tüm diyalog mevcut.
formatted_dialogue = tokenizer.apply_chat_template(
    chat_dialogue,
    tokenize=False,
    add_generation_prompt=False,  # Tam diyalog olduğu için yeni mesaj başlamasını istemiyoruz.
    add_special_tokens=False
)

formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(formatted_dialogue, return_tensors="pt").to("cuda")

# Modelden çıktı al
output = fine_tuned_model.generate(**inputs, max_new_tokens=200, streamer=streamer)

In [ ]:
from transformers import TextStreamer

# Mevcut diyalog: Önceki konuşma (initial conversation)
previous_chat = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."},
    {"role": "user", "content": "I sometimes experience headaches after long work hours."},
    {"role": "assistant", "content": "It might be due to fatigue or dehydration. Ensure you're well-rested and drinking enough water."}
]

# Takip sorusu: Önceki diyaloga dayalı yeni soru
followup_prompt = "Can I take ibuprofen and paracetamol together?"
followup_message = {"role": "user", "content": followup_prompt}

# Önceki konuşma ile takip sorusunu birleştiriyoruz
chat = previous_chat + [followup_message]

# TextStreamer ayarlıyoruz (bu, çıktıyı akışlı görüntülemek içindir)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# ChatML formatına çeviriyoruz; burada add_generation_prompt=True kullanarak modelin asistan yanıtını üretmesini tetikliyoruz.
formatted_prompt = tokenizer.apply_chat_template(
    chat,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize edip modeli GPU'ya taşıyoruz
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

# Modelden çıktı alıyoruz
output = fine_tuned_model.generate(**inputs, max_new_tokens=200, streamer=streamer)


In [ ]:
from transformers import TextStreamer

# Başlangıçta sohbet geçmişini oluşturuyoruz.
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

# Fonksiyon: Sohbet geçmişini kullanarak modelden yanıt al.
def generate_response(new_user_input, chat_history, model, tokenizer, max_new_tokens=200):
    # Kullanıcının yeni mesajını ekle
    chat_history.append({"role": "user", "content": new_user_input})

    # Eğer önceki asistan mesajları varsa, bunlar da chat_history'de yer alır.
    # İsteğe bağlı: Eğer ilk mesajda selamlama varsa, sonraki mesajlarda tekrardan eklemeyebilir veya sistem mesajını sadece bir kez tutabilirsin.

    # ChatML formatına dönüştür:
    formatted_prompt = tokenizer.apply_chat_template(
        chat_history,
        tokenize=False,
        add_generation_prompt=True,  # Yeni asistan yanıtı için prompt ekle
        add_special_tokens=False
    )

    # Tokenize edip modele aktar:
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # TextStreamer ile çıktı akışı:
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Modelden çıktı al
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, streamer=streamer)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Yanıttan sadece asistan kısmını çekmek isteyebilirsin, örneğin:
    # (Burada, modelin ürettiği yanıtın başlangıcında sistem ve kullanıcı mesajları yer alabilir, bu nedenle işleyerek temizleyebilirsin.)
    # Örneğin, "assistant" rolünden sonra gelen kısmı alabilirsin.

    # Sohbet geçmişine asistan yanıtını ekle:
    chat_history.append({"role": "assistant", "content": output_text})

    return output_text

# İlk kullanıcı girişi ve yanıt:
user_input = "Can I take ibuprofen and paracetamol together?"
response = generate_response(user_input, chat_history, fine_tuned_model, tokenizer)
print("Assistant:", response)

# Daha sonra yeni bir takip sorusu:
followup_input = "What should I do if I experience stomach pain after taking them?"
response = generate_response(followup_input, chat_history, fine_tuned_model, tokenizer)
print("Assistant:", response)


In [ ]:
from transformers import TextStreamer
import torch

# Sohbet geçmişini oluşturuyoruz: İlk olarak yalnızca sistem mesajı ekleniyor.
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

def generate_response(new_user_input, chat_history, model, tokenizer, max_new_tokens=200):
    # Kullanıcının yeni mesajını sohbet geçmişine ekle
    chat_history.append({"role": "user", "content": new_user_input})

    # Sohbet geçmişini ChatML formatına dönüştür
    formatted_prompt = tokenizer.apply_chat_template(
        chat_history,
        tokenize=False,
        add_generation_prompt=True,  # Yeni asistan yanıtı üretilmesi için
        add_special_tokens=False
    )

    # Tokenize edip modele aktar
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # TextStreamer, çıktıyı akışlı göstermeyi sağlar (isteğe bağlı)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Modelden çıktı üret
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, streamer=streamer)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Sohbet geçmişine asistan yanıtını ekle
    chat_history.append({"role": "assistant", "content": output_text})

    return output_text

# Sürekli sohbet döngüsü
while True:
    # Kullanıcıdan giriş al
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    # Yeni girdiyle modeli çalıştır
    response = generate_response(user_input, chat_history, fine_tuned_model, tokenizer)

    # Yalnızca chatbotun çıktısını ekrana yazdır
    print("\nOnurMedical Bot:", response, "\n")


In [ ]:
from transformers import TextStreamer
import torch

# Başlangıçta yalnızca bir defa sistem mesajı ekleniyor.
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

def generate_response(new_user_input, chat_history, model, tokenizer, max_new_tokens=200):
    # Yeni kullanıcı mesajını sohbet geçmişine ekle.
    chat_history.append({"role": "user", "content": new_user_input})

    # Eğer sohbet geçmişi ilk turdan (sistem mesajı dahil) fazlaysa,
    # formatlama sırasında sistem mesajını dahil etmiyoruz.
    if len(chat_history) > 2:
        chat_to_format = [msg for msg in chat_history if msg["role"] != "system"]
    else:
        chat_to_format = chat_history

    # ChatML formatına dönüştür
    formatted_prompt = tokenizer.apply_chat_template(
        chat_to_format,
        tokenize=False,
        add_generation_prompt=True,  # Yeni asistan mesajı için gerekli prompt eklenir.
        add_special_tokens=False
    )

    # Tokenize edip modele aktar
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # TextStreamer çıktıyı akışlı göstermek içindir (isteğe bağlı)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Modelden yanıt üret
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, streamer=streamer)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Sohbet geçmişine modelin yanıtını ekle.
    chat_history.append({"role": "assistant", "content": output_text})

    return output_text

# Sürekli sohbet döngüsü: Sadece chatbot çıktısı yazdırılır.
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    response = generate_response(user_input, chat_history, fine_tuned_model, tokenizer)

    # Yalnızca yeni asistan yanıtını ekrana bastırıyoruz.
    print("\nOnurMedical Bot:", response, "\n")


In [ ]:
import torch
import re
from transformers import TextStreamer

# Başlangıçta sistem mesajı var
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

# İlk yanıt mı olduğunu takip etmek için
first_response = True

def generate_response(new_user_input, chat_history, model, tokenizer, first_response, max_new_tokens=200):
    # Kullanıcı mesajını geçmişe ekle
    chat_history.append({"role": "user", "content": new_user_input})

    # Sistem mesajı sadece ilk mesajda kullanılır
    if len(chat_history) > 2:
        chat_to_format = [msg for msg in chat_history if msg["role"] != "system"]
    else:
        chat_to_format = chat_history

    # Prompt formatlama
    formatted_prompt = tokenizer.apply_chat_template(
        chat_to_format,
        tokenize=False,
        add_generation_prompt=True,
        add_special_tokens=False
    )

    # Tokenize et
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # TextStreamer (isteğe bağlı)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Modelden çıktı al
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, streamer=streamer)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    # Eğer ilk yanıttan sonraysa, selamlamayı temizle
    if not first_response:
        output_text = re.sub(
            r"^(Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant\. I understand your concern\.?\s*)",
            "", output_text, flags=re.IGNORECASE
        ).strip()

    # Sohbet geçmişine asistan yanıtı ekle
    chat_history.append({"role": "assistant", "content": output_text})
    return output_text

# Sürekli sohbet döngüsü
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    # İlk yanıt mı diye kontrol et
    is_first_response = len([msg for msg in chat_history if msg["role"] == "assistant"]) == 0

    response = generate_response(user_input, chat_history, fine_tuned_model, tokenizer, is_first_response)

    print("\nOnurMedical Bot:", response, "\n")


In [ ]:
import torch
import re

# Başlangıçta sistem mesajı var
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

def generate_response(new_user_input, chat_history, model, tokenizer, first_response, max_new_tokens=200):
    # Kullanıcı mesajını geçmişe ekle
    chat_history.append({"role": "user", "content": new_user_input})

    # Sistem mesajı sadece ilk mesajda kullanılır
    if len(chat_history) > 2:
        chat_to_format = [msg for msg in chat_history if msg["role"] != "system"]
    else:
        chat_to_format = chat_history

    # Prompt formatlama
    formatted_prompt = tokenizer.apply_chat_template(
        chat_to_format,
        tokenize=False,
        add_generation_prompt=True,
        add_special_tokens=False
    )

    # Tokenize et
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Modelden çıktı al
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    # Eğer ilk yanıttan sonraysa, selamlamayı temizle
    if not first_response:
        output_text = re.sub(
            r"^(Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant\. I understand your concern\.?\s*)",
            "", output_text, flags=re.IGNORECASE
        ).strip()

    # Sohbet geçmişine asistan yanıtı ekle
    chat_history.append({"role": "assistant", "content": output_text})
    return output_text

# Sürekli sohbet döngüsü
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    # İlk yanıt mı diye kontrol et
    is_first_response = len([msg for msg in chat_history if msg["role"] == "assistant"]) == 0

    response = generate_response(user_input, chat_history, fine_tuned_model, tokenizer, is_first_response)

    print("\nOnurMedical Bot:", response, "\n")


In [ ]:
import torch
import re
from transformers import TextStreamer

# Sistem mesajı sadece bir kez eklenecek
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

def generate_response(new_user_input, chat_history, model, tokenizer, is_first_response, max_new_tokens=200):
    # Kullanıcı mesajını geçmişe ekle
    chat_history.append({"role": "user", "content": new_user_input})

    # İlk mesaj değilse sistem mesajını dışla
    if len(chat_history) > 2:
        chat_to_format = [msg for msg in chat_history if msg["role"] != "system"]
    else:
        chat_to_format = chat_history

    # ChatML formatına çevir
    formatted_prompt = tokenizer.apply_chat_template(
        chat_to_format,
        tokenize=False,
        add_generation_prompt=True,
        add_special_tokens=False
    )

    # Tokenize et
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Streamer (ekrana anlık basar, biz ayrıca yazdırmıyoruz)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Yanıtı üret (streamer ekrana basacak)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        streamer=streamer,
        do_sample=False,
        temperature=0.7
    )

    # Modelin cevabını al (yine de geçmişe eklemek için decode edeceğiz)
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    # Sadece yeni asistan mesajını al (input prompt uzunluğundan sonrası)
    input_len = inputs.input_ids.shape[1]
    new_output_ids = output_ids[0][input_len:]
    assistant_response = tokenizer.decode(new_output_ids, skip_special_tokens=True).strip()

    # Eğer ilk mesaj değilse selamlama kısmını regex ile temizle
    if not is_first_response:
        assistant_response = re.sub(
            r"^(Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant\. I understand your concern\.?\s*)",
            "", assistant_response, flags=re.IGNORECASE
        ).strip()

    # Sohbet geçmişine yanıtı ekle
    chat_history.append({"role": "assistant", "content": assistant_response})

# Sohbet döngüsü
while True:
    user_input = input("\nUser: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    is_first = len([m for m in chat_history if m["role"] == "assistant"]) == 0
    generate_response(user_input, chat_history, fine_tuned_model, tokenizer, is_first)


In [ ]:
import torch
import re
from transformers import TextStreamer

# Başlangıçta sadece sistem mesajı var
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

def generate_response(new_user_input, chat_history, model, tokenizer, max_new_tokens=200):
    # Kullanıcı mesajını geçmişe ekle
    chat_history.append({"role": "user", "content": new_user_input})

    # Tüm geçmişi sistem mesajı dahil olacak şekilde formatla
    chat_to_format = chat_history

    # Chat formatına çevir (Mistral destekliyorsa ChatML gibi)
    formatted_prompt = tokenizer.apply_chat_template(
        chat_to_format,
        tokenize=False,
        add_generation_prompt=True,
        add_special_tokens=False
    )

    # Tokenize et
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Streamer: çıktıyı anlık olarak terminale basar
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Model çıktısını al (streamer çıktıyı bastığı için ayrıca print gerekmez)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        streamer=streamer,
        do_sample=False,
        temperature=0.7
    )

    # Assistant cevabını çıkar (chat geçmişine eklemek için)
    input_len = inputs.input_ids.shape[1]
    new_output_ids = output_ids[0][input_len:]
    assistant_response = tokenizer.decode(new_output_ids, skip_special_tokens=True).strip()

    # Selamlamayı kaldır (ilk yanıt değilse)
    is_first_response = len([m for m in chat_history if m["role"] == "assistant"]) == 0
    if not is_first_response:
        assistant_response = re.sub(
            r"^(Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant\. I understand your concern\.?\s*)",
            "", assistant_response,
            flags=re.IGNORECASE
        ).strip()

    # Assistant cevabını geçmişe ekle
    chat_history.append({"role": "assistant", "content": assistant_response})

# Sonsuz sohbet döngüsü
while True:
    user_input = input("\nUser: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    generate_response(user_input, chat_history, fine_tuned_model, tokenizer)


In [ ]:
import torch
import re
from transformers import TextIteratorStreamer
from threading import Thread

# Sistem mesajı
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

def generate_response(new_user_input, chat_history, model, tokenizer, max_new_tokens=200):
    chat_history.append({"role": "user", "content": new_user_input})

    # Chat geçmişini sistem mesajı dahil olarak hazırla
    chat_to_format = chat_history

    formatted_prompt = tokenizer.apply_chat_template(
        chat_to_format,
        tokenize=False,
        add_generation_prompt=True,
        add_special_tokens=False
    )

    # Tokenize
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Streamer'ı başlat
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Model generate işlemi ayrı thread'de başlatılır
    generation_kwargs = dict(
        **inputs,
        max_new_tokens=max_new_tokens,
        streamer=streamer,
        do_sample=False,
        temperature=0.7
    )
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # Stream edilen cevabı topla
    streamed_tokens = ""
    for new_text in streamer:
        streamed_tokens += new_text

    # Assistant cevabını temizle
    is_first_response = len([m for m in chat_history if m["role"] == "assistant"]) == 0
    if not is_first_response:
        streamed_tokens = re.sub(
            r"^(Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant\. I understand your concern\.?\s*)",
            "", streamed_tokens,
            flags=re.IGNORECASE
        ).strip()

    # Temizlenmiş çıktıyı yazdır
    print(f"\nOnurMedical Bot: {streamed_tokens}")

    # Sohbet geçmişine ekle
    chat_history.append({"role": "assistant", "content": streamed_tokens})


# Ana döngü
while True:
    user_input = input("\nUser: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    generate_response(user_input, chat_history, fine_tuned_model, tokenizer)



User: My knee hurts after sports. What do you suggest I do temporarily

OnurMedical Bot: Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant. I understand your concern. You should take a rest for a few days and apply ice packs on the affected area. You can also take painkillers like ibuprofen or paracetamol. If the pain persists, you should consult a doctor and get an X-ray done. Hope I have answered your query. Let me know if I can assist you further. Regards, OnurMedicalBot, a knowledgeable and friendly medical assistant.

User: How do I know if it's a ligament injury?

OnurMedical Bot: If you have a ligament injury, you will have pain, swelling, and difficulty in moving the joint. You should consult a doctor and get an X-ray done. Hope I have answered your query. Let me know if I can assist you further. Regards, OnurMedicalBot, a knowledgeable and friendly medical assistant.

User: what do you reccomend?

OnurMedical Bot: You should take a rest for a few days